# Task 2: Chunking, Embedding, and Vector Store Indexing

This notebook prepares a stratified sample, applies text chunking, generates embeddings for each chunk, and indexes them in a vector store (ChromaDB).

**Checklist:**
1. Stratified sampling
2. Chunking
3. Embedding (MiniLM-L6-v2)
4. Store in ChromaDB
5. Document choices


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np
import random

# Load cleaned dataset
df = pd.read_csv('../data/filtered_complaints.csv')
df['Product'].value_counts()


Product
Credit card        80667
Money transfers     1497
Name: count, dtype: int64

## Stratified Sample: 10,000–15,000 Complaints
Sample proportionally by product category.


In [3]:
# Aim for ~12,000 (adjust size as needed)
SAMPLE_SIZE = 12000

strat_sample, _ = train_test_split(
    df,
    train_size=SAMPLE_SIZE,
    stratify=df['Product'],
    random_state=42
)
print(strat_sample['Product'].value_counts(normalize=True))
strat_sample.reset_index(drop=True, inplace=True)
strat_sample.to_csv('../data/stratified_sample.csv', index=False)


Product
Credit card        0.98175
Money transfers    0.01825
Name: proportion, dtype: float64


## Chunking: Split Narratives
We use a recursive character splitter with chunk_size=500 and overlap=50 (recommended for this use case).


In [4]:
def chunk_text(text, chunk_size=500, chunk_overlap=50):
    text = str(text)
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        if end >= len(text):
            break
        start += chunk_size - chunk_overlap
    return chunks

# Example
chunked_examples = chunk_text(strat_sample['clean_narrative'].iloc[0])
print(chunked_examples)


['old card from bank of america expired and another card was issued but to the wrong old address. when i request the card to be send to my current address they sent a new one with a number different from before. the different number is not a problem but now every transaction gets blocked by fraud department. ive contacted the bank three times, they unlock it but it locks again at the next transaction. when i try to contact the bank they keep doing the same process over and over again but it doesnt', 'the same process over and over again but it doesnt solve the problem. the last time i contacted them today at xxxxxxxx xxxx i told the operator the previous guy did exactly the same thing and he said that theres really nothing much else they can do. i tried to find a different channel to address this issue but couldnt find one on the website. im complaining here so that the complaint can get to the correct department to solve the issue the regular phone doesnt work and the operator also ca

In [5]:
# Apply chunking to all sample narratives
chunk_records = []
for idx, row in strat_sample.iterrows():
    chunks = chunk_text(row['clean_narrative'])
    for chunk_idx, chunk in enumerate(chunks):
        chunk_records.append({
            'complaint_id': row['Complaint ID'],
            'product': row['Product'],
            'issue': row['Issue'],
            'company': row['Company'],
            'state': row['State'],
            'date_received': row['Date received'],
            'chunk_index': chunk_idx,
            'total_chunks': len(chunks),
            'chunk_text': chunk
        })

chunk_df = pd.DataFrame(chunk_records)
chunk_df.to_csv('../data/stratified_sample_chunked.csv', index=False)
chunk_df.head()


,complaint_id,product,issue,company,state,date_received,chunk_index,total_chunks,chunk_text
0,8769190,Credit card,Problem when making payments,"BANK OF AMERICA, NATIONAL ASSOCIATION",MA,2024-04-15,0,3,old card from bank of america expired and anot...
1,8769190,Credit card,Problem when making payments,"BANK OF AMERICA, NATIONAL ASSOCIATION",MA,2024-04-15,1,3,the same process over and over again but it do...
2,8769190,Credit card,Problem when making payments,"BANK OF AMERICA, NATIONAL ASSOCIATION",MA,2024-04-15,2,3,r phone doesnt work and the operator also cant...
3,1773833,Credit card,Billing disputes,"CITIBANK, N.A.",CA,2016-02-04,0,1,i have a bill from macys that i am late on. th...
4,2056598,Credit card,Transaction issue,SYNCHRONY FINANCIAL,VA,2016-08-10,0,2,paypal has a repeated poor business practice o...


## Embedding: MiniLM-L6-v2
We use the sentence-transformers library to embed each chunk.


In [6]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(chunk_df['chunk_text'].tolist(), show_progress_bar=True, batch_size=64)
print(f"Embedding shape: {embeddings.shape}")


Batches: 100%|██████████| 538/538 [01:47<00:00,  5.01it/s]


Embedding shape: (34394, 384)


## Vector Store: Index with ChromaDB
Store embeddings and metadata for semantic search.


In [7]:
import chromadb
from chromadb.config import Settings
from tqdm import tqdm

db = chromadb.Client(Settings(persist_directory="../vector_store"))
collection = db.get_or_create_collection('complaints_sample')

# Add chunks and metadata to ChromaDB
for i in tqdm(range(len(chunk_df))):
    collection.add(
        embeddings=[embeddings[i]],
        documents=[chunk_df['chunk_text'].iloc[i]],
        metadatas=[{
            'complaint_id': chunk_df['complaint_id'].iloc[i],
            'product': chunk_df['product'].iloc[i],
            'issue': chunk_df['issue'].iloc[i],
            'company': chunk_df['company'].iloc[i],
            'state': chunk_df['state'].iloc[i],
            'date_received': chunk_df['date_received'].iloc[i],
            'chunk_index': chunk_df['chunk_index'].iloc[i],
            'total_chunks': chunk_df['total_chunks'].iloc[i],
        }],
        ids=[f"{chunk_df['complaint_id'].iloc[i]}_{chunk_df['chunk_index'].iloc[i]}"]
    )
print("Embedding & chunk data stored in ChromaDB.")


Failed to send telemetry event client_start: capture() takes 1 positional argument but 3 were given
Using embedded DuckDB without persistence: data will be transient


No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
  0%|          | 0/34394 [00:00<?, ?it/s]


ValueError: Expected embeddings to be a list, got [array([-2.15353426e-02, -6.83808774e-02, -3.29628550e-02, -3.66154462e-02,
       -5.15383221e-02, -8.83623660e-02,  4.35030386e-02, -1.11456692e-01,
        8.92012343e-02, -3.63409892e-02,  2.97503583e-02,  7.70042911e-02,
        2.66289245e-03, -2.67017796e-03, -1.19301174e-02,  5.28362673e-03,
       -7.35860392e-02,  6.95732236e-03, -3.80913913e-02,  6.68813884e-02,
        2.86225975e-02,  2.76979189e-02, -8.94014761e-02,  4.81026843e-02,
       -2.56231353e-02, -5.37405610e-02, -5.74529432e-02,  1.23508992e-02,
       -2.66126506e-02, -7.47334212e-02,  2.76590120e-02,  8.83178562e-02,
       -7.30865682e-03, -4.32395190e-02,  2.26176474e-02, -3.18315253e-02,
       -5.23733273e-02,  4.74348404e-02, -2.59456523e-02,  1.66992452e-02,
       -2.60761436e-02,  4.33442071e-02,  7.96724204e-03, -3.86025049e-02,
        7.01640593e-03,  1.96517892e-02, -2.33956017e-02,  1.19415164e-01,
        1.14054933e-01,  6.98195845e-02, -4.97977

## Summary
- Sampling is stratified by product for representativeness.
- Chunking uses 500 char size, 50 overlap for optimal embedding context.
- Embeddings use all-MiniLM-L6-v2 (fast & accurate for semantic search).
- ChromaDB is used for efficient neural search on internal complaint data.
